In [2]:
import pandas as pd

In [ ]:
# Carga el excel de compras totales
compras_totales = pd.read_excel('ct_py.xlsx')

In [ ]:
# Carga el excel de compras semanaes
compras_semanales = pd.read_excel('cs_py.xlsx')

In [ ]:
# modificar campos antes de meter
compras_semanales['PC.UNIT'] = compras_semanales['TOTAL'] / compras_semanales['CANTIDAD']
compras_semanales['AÑO'] = pd.DatetimeIndex(compras_semanales['FECHA']).year
compras_semanales['DÍA'] = pd.DatetimeIndex(compras_semanales['FECHA']).day
compras_semanales['MES'] = pd.DatetimeIndex(compras_semanales['FECHA']).month
compras_semanales['FECHA'] = pd.to_datetime(compras_semanales['FECHA'])
compras_semanales['MES.TEXTO'] = compras_semanales['FECHA'].dt.strftime('%B')



In [ ]:
print(compras_semanales.head())

In [ ]:
# Guardamos los cambios excel cs
compras_semanales.to_excel('cs_py.xlsx', index=False)

In [ ]:
# volver a cargarlo
compras_semanales = pd.read_excel('cs_py.xlsx')

In [ ]:
# Meter cs a ct
compras_totales = pd.concat([compras_totales, compras_semanales], ignore_index=True)


In [ ]:
# Guardamos el excel nuevo para la semana próxima
compras_totales.to_excel('ct_py.xlsx', index=False)

# Crear el UPC

In [ ]:
compras_totales = pd.read_excel('ct_py.xlsx')

In [ ]:
# Ordenar el DataFrame por REFERENCIA y FECHA
ct_py = compras_totales.sort_values(by=['REFERENCIA', 'FECHA'])


In [ ]:
# Obtener el último precio de compra para cada referencia de producto
upc = ct_py.groupby('REFERENCIA').last()[['PC.UNIT']].reset_index()

In [ ]:
ruta_guardar = r'F:\VALIJA DIGITAL\GRS\analisis\VSC\upc.xlsx'

In [ ]:
# crear el excel upc
upc.to_excel(ruta_guardar, index=False)

# Cargar ventas semanales y realizar los cálculos en los campos

In [64]:
# Cargar el excel de clientes y comerciales
clientes = pd.read_excel('CLIENTES.xlsx')
comerciales = pd.read_excel('comerciales.xlsx')

In [65]:
# Carga el excel de ventas semanaes
ventas_semanales = pd.read_excel('vs_py.xlsx')

nuevos campos: FACTURA, DIA, MES, MES.TXT, AÑO, D.SEM, SEMANA, TRIMESTRE, CLIENTE, COMERCIAL, VENDIDO POR, PC.UNIT, PC.TTL, BENEFICIO, M, PROVEEDOR

In [66]:
print(ventas_semanales.columns)
print(clientes.columns)
print(comerciales.columns)

Index(['REFERENCIA', 'DESCRIPCION', 'COD.CLIENTE', 'CANTIDAD', 'PVP', 'DTO.V',
       'TTL', 'SERIE', 'NFAC', 'FECHA', 'COD.COMERCIAL', 'FAMILIA'],
      dtype='object')
Index(['COD.COMER', 'COD.CLIENTE', 'R.SOCIAL', 'NOMBRE COMERCIAL'], dtype='object')
Index(['COD', 'VENDIDO POR'], dtype='object')


In [67]:
print(ventas_semanales.head())

  REFERENCIA        DESCRIPCION  COD.CLIENTE  CANTIDAD     PVP  DTO.V     TTL  \
0        NaN          ENVÍO MRW            1       1.0    8.26    0.0    8.26   
1        NaN        INSTALACIÓN            1       1.0   90.00    0.0   90.00   
2        NaN        INSTALACION            1       3.0   66.00    0.0  198.00   
3        NaN   PORTES PROVEEDOR            1       1.0   69.00    0.0   69.00   
4        NaN  INSTALACION TOLDO            1       1.0  110.00    0.0  110.00   

  SERIE  NFAC       FECHA  COD.COMERCIAL FAMILIA  
0   WEB     4  22/05/2024           34.0     NaN  
1    OV  3019  20/05/2024           34.0     NaN  
2    OV  3020  20/05/2024           34.0     NaN  
3    OV  3034  21/05/2024            NaN     NaN  
4  ABOV    73  23/05/2024            NaN     NaN  


In [68]:
# Concatenar SERIE-NFAC
ventas_semanales['FACTURA'] = ventas_semanales['SERIE'].astype(str) + '-' + ventas_semanales['NFAC'].astype(str)


In [69]:
# FECHA
ventas_semanales['FECHA'] = pd.to_datetime(ventas_semanales['FECHA'], dayfirst=True)

# Crear nuevas columnas con partes específicas de la fecha
ventas_semanales['DIA'] = ventas_semanales['FECHA'].dt.day
ventas_semanales['MES'] = ventas_semanales['FECHA'].dt.month
ventas_semanales['MES.TXT'] = ventas_semanales['FECHA'].dt.strftime('%B')
ventas_semanales['AÑO'] = ventas_semanales['FECHA'].dt.year
ventas_semanales['D.SEM'] = ventas_semanales['FECHA'].dt.strftime('%A')
ventas_semanales['SEMANA'] = ventas_semanales['FECHA'].dt.isocalendar().week
ventas_semanales['TRIMESTRE'] = ventas_semanales['FECHA'].dt.quarter

# Verificar que las nuevas columnas se han creado correctamente
# print(ventas_semanales.head())

In [70]:
# Left join para traer la R.SOCIAL de la tabla clientes
ventas_semanales = ventas_semanales.merge(clientes[['COD.CLIENTE', 'R.SOCIAL']], how='left', on='COD.CLIENTE')

# la columna R.SOCIAL como CLIENTE
ventas_semanales['CLIENTE'] = ventas_semanales['R.SOCIAL']

In [71]:
#Eliminar la columna R.SOCIAL
ventas_semanales.drop(columns=['R.SOCIAL'], inplace=True)

In [72]:
# Left join para traer el COMERCIAL de la tabla CLIENTES
ventas_semanales = ventas_semanales.merge(clientes[['COD.CLIENTE', 'NOMBRE COMERCIAL']], how='left', on='COD.CLIENTE')

# Renombrar la columna como COMERCIAL
ventas_semanales.rename(columns={'NOMBRE COMERCIAL': 'COMERCIAL'}, inplace=True)

In [73]:
print(ventas_semanales['COMERCIAL'].unique())

['LF' 'Arnaldo Lopez' 'Angel JR' 'Eduardo Gómez' 'Marta Soto'
 'Mónica Rodriguez ' 'Carolina Garcia' 'Gustavo Loarce' 'Jennifer Perez']


In [74]:
# Left join para traer el VENDIDO POR de la tabla comerciales
ventas_semanales = ventas_semanales.merge(comerciales[['COD', 'VENDIDO POR']], how='left', left_on='COD.COMERCIAL', right_on='COD')

In [75]:
#Eliminar la columna COD
ventas_semanales.drop(columns=['COD'], inplace=True)

In [76]:
# Actualizar valores NaN en 'VENDIDO POR' con 'LF'
ventas_semanales['VENDIDO POR'].fillna('LF', inplace=True)


C:\Users\GonzaloRS\AppData\Local\Temp\ipykernel_17404\2108872507.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ventas_semanales['VENDIDO POR'].fillna('LF', inplace=True)


In [77]:
print(ventas_semanales['VENDIDO POR'].unique())

['Gonzalo Robles' 'LF' 'Angel JR' 'Jennifer Perez' 'Marta Soto'
 'Mónica Rodriguez ' 'Carolina Garcia' 'Gustavo Loarce']


In [78]:
# pc.unit // cargar el excel
upc = pd.read_excel('upc.xlsx')

In [79]:
upc.rename(columns={'REFERENCIA': 'ID_REF', 'PC.UNIT': 'PRECIO_U'}, inplace=True)

In [80]:
# traer el PC.UNIT segun REFERENCIA
ventas_semanales = ventas_semanales.merge(upc[['ID_REF', 'PRECIO_U']], how='left', left_on='REFERENCIA', right_on='ID_REF')

In [81]:
#Eliminar la columna ID_REF y renombrar la columna PRECIO_U
ventas_semanales.drop(columns=['ID_REF'], inplace=True)
ventas_semanales.rename(columns={'PRECIO_U': 'PC.UNIT'}, inplace=True)


In [82]:
print(ventas_semanales.columns)

Index(['REFERENCIA', 'DESCRIPCION', 'COD.CLIENTE', 'CANTIDAD', 'PVP', 'DTO.V',
       'TTL', 'SERIE', 'NFAC', 'FECHA', 'COD.COMERCIAL', 'FAMILIA', 'FACTURA',
       'DIA', 'MES', 'MES.TXT', 'AÑO', 'D.SEM', 'SEMANA', 'TRIMESTRE',
       'CLIENTE', 'COMERCIAL', 'VENDIDO POR', 'PC.UNIT'],
      dtype='object')


In [83]:
# Actualizar valores NaN en 'PC.UNIT' con 'PVP/2'
ventas_semanales['PC.UNIT'].fillna(ventas_semanales.apply(lambda row: row['PVP'] / 2 if pd.isnull(row['PC.UNIT']) and row['FAMILIA'] == 45 else row['PC.UNIT'], axis=1), inplace=True)


C:\Users\GonzaloRS\AppData\Local\Temp\ipykernel_17404\3128837863.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ventas_semanales['PC.UNIT'].fillna(ventas_semanales.apply(lambda row: row['PVP'] / 2 if pd.isnull(row['PC.UNIT']) and row['FAMILIA'] == 45 else row['PC.UNIT'], axis=1), inplace=True)


In [84]:
filtro_familia_45 = ventas_semanales['FAMILIA'] == 45
columnas_interesantes = ['PVP', 'PC.UNIT']

print(ventas_semanales.loc[filtro_familia_45, columnas_interesantes])

        PVP  PC.UNIT
6     15.00    7.500
7     42.00   21.000
8     20.00   10.000
9     15.00    7.500
10    16.50    8.250
11    20.00   10.000
12    16.50    8.250
13   650.00  325.000
14   250.00  125.000
15    80.00   40.000
17   550.00  275.000
18   250.00  125.000
19   550.00  275.000
22    20.00   10.000
30     2.34    1.170
31     5.20    2.600
32     1.95    0.975
133    0.77    0.385
157    3.70    1.000
162    3.38    1.300
163    3.30    1.625
177   15.00    7.500
182   13.40    5.270
183   11.00    5.400
230   11.00    3.050
638   20.00    6.760


In [85]:
print(ventas_semanales.columns)

Index(['REFERENCIA', 'DESCRIPCION', 'COD.CLIENTE', 'CANTIDAD', 'PVP', 'DTO.V',
       'TTL', 'SERIE', 'NFAC', 'FECHA', 'COD.COMERCIAL', 'FAMILIA', 'FACTURA',
       'DIA', 'MES', 'MES.TXT', 'AÑO', 'D.SEM', 'SEMANA', 'TRIMESTRE',
       'CLIENTE', 'COMERCIAL', 'VENDIDO POR', 'PC.UNIT'],
      dtype='object')


In [86]:
ventas_semanales['PC.UNIT']=ventas_semanales['PC.UNIT'].astype(float)
ventas_semanales['CANTIDAD']=ventas_semanales['CANTIDAD'].astype(float)
ventas_semanales['TTL']=ventas_semanales['TTL'].astype(float)

In [87]:
ventas_semanales['PC.TTL']= ventas_semanales['PC.UNIT'] * ventas_semanales['CANTIDAD']


In [88]:
ventas_semanales['BENEFICIO']= ventas_semanales['TTL'] - ventas_semanales['PC.TTL']


In [89]:
ventas_semanales['M'] = round((ventas_semanales['TTL'] / ventas_semanales['PC.TTL'] -1)*100,2)

In [92]:

ruta_guardar = r'F:\VALIJA DIGITAL\GRS\analisis\VSC\vs_py.xlsx'

ventas_semanales.to_excel(ruta_guardar, index=False)


# Cargar a ventas totales

In [91]:
# Carga el excel de ventas totales
ventas_totales = pd.read_excel('vt_py.xlsx')

In [93]:
# Metemos las nuevas ventas en las totales.
ventas_totales = pd.concat([ventas_totales, ventas_semanales], ignore_index=True)

In [94]:
ruta2 = r'F:\VALIJA DIGITAL\GRS\analisis\VSC\prueba.xlsx'

In [95]:
# Guardamos el excel nuevo para la semana próxima
ventas_totales.to_excel(ruta2, index=False)

# ENCHUFAR A SQL UNA TABLA HECHA CON EXCEL DESDE PYTHON

In [124]:
import pandas as pd
import sqlalchemy
import pyodbc

In [131]:
# Configurar la conexión a SQL Server
server = 'PRA_GENERAL\SQLEXPRESS'
database = 'Prueba DB'
conn_str = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = sqlalchemy.create_engine(conn_str)

In [132]:
# Conexión con pyodbc para ejecutar la consulta de eliminación
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes')
cursor = conn.cursor()


In [127]:
# Nombre de la tabla en la base de datos
tabla_sql = 'dbo.hola_desde_python'

In [133]:
# Eliminar la tabla si existe
drop_table_query = f"""
IF OBJECT_ID('{tabla_sql}', 'U') IS NOT NULL
DROP TABLE {tabla_sql};
"""
cursor.execute(drop_table_query)
conn.commit()
cursor.close()
conn.close()

In [129]:
# Cargar los datos desde el archivo Excel
df = pd.read_excel('prueba_sql.xlsx')

In [111]:
# Definir los tipos de datos para cada columna
dtype_mapping = {
    'REFERENCIA': sqlalchemy.types.NVARCHAR(length=255),
    'DESCRIPCION': sqlalchemy.types.NVARCHAR(length=255),
    'COD.CLIENTE': sqlalchemy.types.Float,
    'CANTIDAD': sqlalchemy.types.Float,
    'PVP': sqlalchemy.types.Float,
    'DTO.V': sqlalchemy.types.Float,
    'TTL': sqlalchemy.types.Float,
    'SERIE': sqlalchemy.types.NVARCHAR(length=255),
    'NFAC': sqlalchemy.types.Float,
    'FECHA': sqlalchemy.types.DateTime,
    'COD.COMERCIAL': sqlalchemy.types.Float,
    'FAMILIA': sqlalchemy.types.NVARCHAR(length=255),
    'FACTURA': sqlalchemy.types.NVARCHAR(length=255),
    'DIA': sqlalchemy.types.Float,
    'MES': sqlalchemy.types.Float,
    'MES.TXT': sqlalchemy.types.NVARCHAR(length=255),
    'AÑO': sqlalchemy.types.Float,
    'D.SEM': sqlalchemy.types.NVARCHAR(length=255),
    'SEMANA': sqlalchemy.types.Float,
    'TRIMESTRE': sqlalchemy.types.Float,
    'CLIENTE': sqlalchemy.types.NVARCHAR(length=255),
    'COMERCIAL': sqlalchemy.types.NVARCHAR(length=255),
    'VENDIDO POR': sqlalchemy.types.NVARCHAR(length=255),
    'PC.UNIT': sqlalchemy.types.Float,
    'PC.TTL': sqlalchemy.types.Float,
    'BENEFICIO': sqlalchemy.types.Float,
    'MARGEN': sqlalchemy.types.Float
}


In [112]:

# Crear la tabla en la base de datos SQL Server con los datos del DataFrame
df.to_sql(name=tabla_sql, con=engine, if_exists='replace', index=False, dtype=dtype_mapping)
